In [3]:
import json
import os
import numpy as np
import pandas as pd
from datasets import load_dataset
from urllib.parse import parse_qs, urlparse
import requests
import chromadb
import openai
from chromadb.config import Settings
import os
import openai
from openai import OpenAI
import chromadb
import fitz  # PyMuPDF
from chromadb.utils import embedding_functions
import chromadb.utils.embedding_functions as embedding_functions
from chromadb.config import Settings

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from langchain_community.vectorstores import Chroma
import base64
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import os
from langchain.evaluation.qa import QAEvalChain
from bert_score import score
from openai import OpenAI

In [5]:
from langchain.text_splitter import CharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_openai import ChatOpenAI
from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders import PyPDFLoader
from langchain.prompts import PromptTemplate
#from langchain_community.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
import base64
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import os
from langchain.evaluation.qa import QAEvalChain
from bert_score import score
from openai import OpenAI

In [7]:
import base64


def extract_pdf_url(url):
    """
    Extracts the actual PDF URL from the given URL.
    Decodes it from base64 if necessary.
    """
    if url.lower().endswith('.pdf'):
        return url  # Direct PDF URL
    else:
        parsed_url = urlparse(url)
        query_params = parse_qs(parsed_url.query)
        pdf_target = query_params.get('pdfTarget', [None])[0]

        if pdf_target:
            pdf_url = base64.b64decode(pdf_target).decode('utf-8')
            return pdf_url
        else:
            raise ValueError("No valid PDF URL found in the provided URL")

def download_pdf(url, save_path):
    """
    Downloads a PDF from a given URL.
    """
    try:
        pdf_url = extract_pdf_url(url)
        response = requests.get(pdf_url, stream=True)
        response.raise_for_status()  # Ensure the request was successful

        with open(save_path, 'wb') as file:
            for chunk in response.iter_content(chunk_size=8192):
                file.write(chunk)

        print(f"Downloaded PDF from: {pdf_url} to {save_path}")
    except Exception as e:
        print(f"Error downloading PDF: {e}")
        
def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page_num in range(len(doc)):
        page = doc.load_page(page_num)
        text += page.get_text()
    return text

In [65]:
def create_knowledge_hub(pdf_path, doc_name):
    # Normalize doc_name to create a valid directory name
    normalized_doc_name = doc_name.replace(' ', '_').replace('/', '_')
    
    # Define the base directory for vector databases
    base_dir = "vector_db"
    openai_ef = embedding_functions.OpenAIEmbeddingFunction(api_key=openai_api_key)
    # Ensure the base directory exists
    if not os.path.exists(base_dir):
        os.makedirs(base_dir)

    # Define the full path to the specific database directory
    db_directory = os.path.join(base_dir, "db_" + normalized_doc_name)
    embeddings = OpenAIEmbeddings(api_key=openai_api_key)
    # Check if the database directory already exists
    if os.path.exists(db_directory):
        print(f"Using existing database for document: {doc_name}")
        vectordb = Chroma(persist_directory=db_directory, embedding_function=embeddings)
    else:
        print(f"Creating new database for document: {doc_name}")

        loader = PyPDFLoader(pdf_path)
        documents = loader.load()

        splitter = CharacterTextSplitter(
            chunk_size=1300, 
            chunk_overlap=5,
            #separators=["\n\n", "\n", " ", ""],
            length_function=len)
        texts = splitter.split_documents(documents)
        print(len(texts))
        vectordb = Chroma.from_documents(
            texts, 
            embeddings,  
            persist_directory=db_directory,
            )

        vectordb.persist()

    return vectordb

In [59]:
def query_openai_with_context(query, vectordb, openai_api_key, top_k=5):
    # Initialize Chroma client

    #context = vectordb.similarity_search(query, k = 2)
    context = vectordb.as_retriever()
    # retrieved_docs = vectordb.similarity_search(query, k=2)
    # print(retrieved_docs[0].page_content)
    # context = "".join(doc.page_content + "\n" for doc in retrieved_docs)
    print(context)

    #openai_ef = embedding_functions.OpenAIEmbeddingFunction(api_key=openai_api_key)
    
    template = """You are a financial chatbot trained to answer questions based on the information provided in 10-K
    documents. Your responses should be directly sourced from the content of these documents. When asked
    a question, ensure that your answer is explicitly supported by the text in the 10-K filing, and do not
    include any external information, interpretations, or assumptions not clearly stated in the document. If
    a question pertains to financial data or analysis that is not explicitly covered in the 10-K filing provided,
    respond by stating that the information is not available in the document. Your primary focus should
    be on accuracy, specificity, and adherence to the information in 10-K documents, particularly regarding
    financial statements, company performance, and market position."""
    
    prompt = f"\nContext:\n{context}\n\nQuery: {query}\n\nAnswer:"


    messages = [
            {"role": "system", "content": template},
            {"role": "user", "content": prompt}
        ]
    response = get_assistant_response(messages)
    
    return response

In [60]:
def display_chat_history(messages):
    for message in messages:
        print(f"{message['role'].capitalize()}: {message['content']}")

def get_assistant_response(messages):
  client = OpenAI(api_key = openai_api_key)
  response = client.chat.completions.create(
      model="gpt-3.5-turbo",
      messages=[{"role": m["role"], "content": m["content"]} for m in messages],
  )

  return response.choices[0].message.content

In [61]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def calculate_cosine_similarity(text1, text2):
    # Create a TfidfVectorizer
    vectorizer = TfidfVectorizer()

    # Fit and transform the two texts
    tfidf_matrix = vectorizer.fit_transform([text1, text2])

    # Calculate cosine similarity
    cosine_sim = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:2])

    # Extract the similarity score
    similarity_score = cosine_sim[0][0]
    
    return similarity_score


def calculate_bertscore(candidate, reference):
    P, R, F1 = score([candidate], [reference], lang="en", verbose=True)
    return P.mean().item()

In [62]:
def evaluate_llm_responses(question, model_answer, refrence_answer):
    for question in question:
        messages = [
            {"role": "system", "content": "You are an assistant that provides concise and accurate answers."},
            {"role": "user", "content": question}
        ]
        response = model_answer

    evaluation_scores = []
    for i in range(len(response)):
        evaluation_prompt = f"""
        Evaluate the following response against the reference answer. Assign a score between 0 and 1 based on correctness and provide a brief justification.

        Question: {question}
        Response: {model_answer}
        Reference Answer: {refrence_answer}

        Score (0 to 1):
        Justification:
        """
        messages = [
            {"role": "system", "content": "You are an evaluator that scores responses based on correctness."},
            {"role": "user", "content": evaluation_prompt}
        ]
        evaluation_response = get_assistant_response(messages)

        evaluation_text = evaluation_response.strip()
        try:
            score_line = evaluation_text.split('\n')[0]
            score = float(score_line.split(':')[1].strip())
            evaluation_scores.append(score)
        except Exception as e:
            print(f"Error parsing score: {e}")
            evaluation_scores.append(0.0)

    average_score = sum(evaluation_scores) / len(evaluation_scores) if evaluation_scores else 0
    print(f'Average Correctness Score: {average_score:.2f}')
    return average_score

In [66]:
def main(): 
    dataset = load_dataset("PatronusAI/financebench")
    df = pd.DataFrame(dataset['train'])
    test = df[:5]
    results_list = []

    for index, row in test.iterrows():
        download_dir = "pdf_documents"
        os.makedirs(download_dir, exist_ok=True)
        doc_link = row['doc_link']
        doc_name = row['doc_name']
        question = row['question']
        ref_answer = row['answer']
        ref_context = row['evidence_text']
        doc_path = os.path.join(download_dir, f"{doc_name}.pdf")

        download_pdf(doc_link, doc_path)
        vector_db = create_knowledge_hub(doc_path, doc_name)
        print("Querying Model now")
        model_answer = query_openai_with_context(question, vector_db, openai_api_key = openai_api_key)
        print(model_answer)

        # Evaluation for structured QA 
        cosine_similarity_score = calculate_cosine_similarity(model_answer, ref_answer)
        bert_score = calculate_bertscore(model_answer, ref_answer)
        llm_eval = evaluate_llm_responses(question, model_answer, ref_answer)

        # Append results to the list
        results_list.append({
            'doc_name': doc_name,
            'question': question,
            'ref_answer': ref_answer,
            'model_answer': model_answer,
            'cosine_similarity': cosine_similarity_score,
            'bert_score': bert_score,
            'llm_eval': llm_eval
        })

    # Convert the list of dictionaries to a DataFrame
    results_df = pd.DataFrame(results_list)

    # Save results to CSV
    results_df.to_csv('results2.csv', index=False)

In [67]:
main()

Downloaded PDF from: https://investors.3m.com/financials/sec-filings/content/0001558370-19-000470/0001558370-19-000470.pdf to pdf_documents\3M_2018_10K.pdf
Using existing database for document: 3M_2018_10K
Querying Model now
tags=['Chroma', 'OpenAIEmbeddings'] vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x0000027B1CFA2A50>
The FY2018 capital expenditure amount for 3M was approximately $1.417 billion as per the Consolidated Statements of Cash Flows in the 10-K filing.


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


calculating scores...
computing bert embedding.


100%|██████████| 1/1 [00:00<00:00,  4.14it/s]


computing greedy matching.


...

calculating scores...
computing bert embedding.


100%|██████████| 1/1 [00:00<00:00,  2.07it/s]


computing greedy matching.


100%|██████████| 1/1 [00:00<?, ?it/s]


done in 0.49 seconds, 2.03 sentences/sec
Average Correctness Score: 0.02
